In [5]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import string
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from keras.models import Sequential
from keras.layers import Embedding, Conv1D, MaxPooling1D, LSTM, Dense

# Step 1: Load the data from the CSV file
data_path = '/Users/charansaisadla/Desktop/ub classes/SEM2/NLP/data project/'
data = pd.read_csv(data_path + "input.csv")

texts = data['text'].tolist()
classes = data['class'].tolist()

# Step 2: Convert the text data into numerical representations
nltk.download('punkt')
nltk.download('stopwords')

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

tokenized_texts = []
for text in texts:
    tokens = [lemmatizer.lemmatize(token.lower()) for token in word_tokenize(text) if
              token.lower() not in stop_words and token.lower() not in string.punctuation]
    tokenized_texts.append(tokens)

tokenizer = Tokenizer()
tokenizer.fit_on_texts(tokenized_texts)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(tokenized_texts)
max_sequence_length = 500
padded_sequences = pad_sequences(sequences, maxlen=max_sequence_length)

# Step 3: Encode the class labels
label_encoder = LabelEncoder()
encoded_classes = label_encoder.fit_transform(classes)
num_classes = len(label_encoder.classes_)
one_hot_classes = to_categorical(encoded_classes, num_classes=num_classes)

# Step 4: Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(padded_sequences, one_hot_classes, test_size=0.2, random_state=42)

# Step 5: Define and train the CRNN model
model = Sequential()
model.add(Embedding(len(word_index) + 1, 100, input_length=max_sequence_length))
model.add(Conv1D(256, 8, activation='relu'))
model.add(MaxPooling1D(pool_size=5))
model.add(LSTM(128))
model.add(Dense(num_classes, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=5, batch_size=256)

# Step 6: Evaluate the model on the test set
y_pred_prob = model.predict(X_test)
y_pred = y_pred_prob.argmax(axis=1)
y_test_encoded = y_test.argmax(axis=1)
report = classification_report(y_test_encoded, y_pred)
print(report)


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/charansaisadla/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/charansaisadla/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Epoch 1/5
37/37 [==============================] - 29s 758ms/step - loss: 1.8720 - accuracy: 0.3195 - val_loss: 1.2349 - val_accuracy: 0.5183
Epoch 2/5
37/37 [==============================] - 27s 736ms/step - loss: 0.7747 - accuracy: 0.7116 - val_loss: 0.6776 - val_accuracy: 0.7396
Epoch 3/5
37/37 [==============================] - 29s 772ms/step - loss: 0.3448 - accuracy: 0.8806 - val_loss: 0.5612 - val_accuracy: 0.8005
Epoch 4/5
37/37 [==============================] - 30s 803ms/step - loss: 0.1352 - accuracy: 0.9577 - val_loss: 0.5708 - val_accuracy: 0.8120
Epoch 5/5
74/74 [==============================] - 3s 38ms/step
              precision    recall  f1-score   support

           0       0.95      0.93      0.94       214
           1       0.99      1.00      0.99       217
           2       0.64      0.67      0.65       213
           3       0.88      0.90      0.89       431
           4       1.00      0.94      0.97       215
           5       0.93      0.97      0.95